In [ ]:
%matplotlib qt
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (12, 5)
# plt.rcParams.update({"font.size": 14})
import numpy as np
from xbout import open_boutdataset

In [ ]:
#ds_hires = ds.copy(deep=True)

In [ ]:
ds = open_boutdataset(chunks={"t": 4},datapath="/Users/power8/Documents/01_code/04_bout/churning_mode/data/BOUT.dmp.*.nc")
# ds = open_boutdataset(chunks={"t": 4},datapath="/Users/power8/Documents/01_code/04_bout/running_churn/data/BOUT.dmp.*.nc")

#Use squeeze() to get rid of the y-dimension, which has length 1 as blob2d does not
#simulate the parallel dimension.

dx = ds["dx"].isel(y=0).values
dy = ds["dy"].isel(x=0).values

ds = ds.squeeze(drop=True)

# Get rid of existing "x" coordinate, which is just the index values.
x = "x"
y = "y"
ds = ds.drop("x")
ds = ds.drop("y")
# # # Create a new coordinate, which is length in units of rho_s
ds = ds.assign_coords(x=np.arange(ds.sizes[x])*dx)
ds = ds.assign_coords(y=np.arange(ds.sizes["y"])*dy)

ngc = int((len(ds["x"]) - len(ds["y"]))/2)
ds = ds.isel(x=range(ngc,len(ds["x"])-ngc))

# Original

In [ ]:
##### ANIMATE LIST OF VARIABLES
ds.bout.animate_list(["omega", "P", "psi", "phi", "u_x", "u_y"])
# ds.bout.animate_list(["P", "omega"], vmin=0.5, vmax=1.5,aspect='equal')
# ds.bout.animate_list(["psi"])
#ds.bout.animate_list([ds["P"] * ds["x"]])

In [ ]:
##### PLOT COLOURMAPS OF SEVERAL VARIABLES AT GIVEN TIMESTAMP #####
t = -1
vars = ["P", "psi", "omega", "phi"]
fig,ax = plt.subplots(ncols=len(vars))
for i,v in enumerate(vars):
    if v == "P" or v == "psi":
        vmin = ds[v][0].values.min()
        vmax = ds[v][0].values.max()
    else:
        vmin = None 
        vmax = None
    ax[i].contourf(ds[x],ds[y],ds[v][t].values.T, vmin=vmin, vmax = vmax,levels=20)
    ax[i].set_xlabel(x)
    ax[i].set_ylabel(y)
    ax[i].set_title(v)
fig.tight_layout()

In [ ]:
##### ANIMATE MIXED COLOURMAPS AND LINE PLOTS #####
offset = -14
ngc = 3
nx = 32 
ny = 32
ds.bout.animate_list(["phi",
                      "omega",
                      ds["phi"].isel(y=int(ny/2) + offset), 
                      ds["omega"].isel(y=int(nx/2) + offset),
                      ds["phi"].isel(x=int(ny/2) + ngc + offset), 
                      ds["omega"].isel(x=int(nx/2) + ngc + offset),
                      ],
                      ncols=2,
                    )

In [ ]:
##### ANIMATE CONTOUR PLOTS OF PSI #####
from matplotlib import animation
# Generate grid for plotting
xmin = 0
xmax = -1
xvals = ds[x][xmin:xmax]
yvals = ds[y][xmin:xmax]
v = "P"
var = ds[v].values[:,xmin:xmax,xmin:xmax]

fig = plt.figure()
ax = plt.axes(xlim=(0, xvals.values.max()), ylim=(0, yvals.values.max()),aspect='equal')  
vmin = var[0].min()
vmax = var[0].max()
levels = np.sort(list(np.linspace(vmin,vmax,20)) + [0])

# animation function
plot_every = 5
def animate(i): 
    ax.clear()
    z = var[plot_every*i].T
    cont = ax.contourf(xvals, yvals, z, vmin=vmin, vmax=vmax, levels=levels)
    if (i == 0):
        ax.set_title(plot_every*i)
    else:
        ax.set_title(plot_every*i)
    ax.set_xlabel(x)
    ax.set_ylabel(y)

    return cont  

anim = animation.FuncAnimation(fig, animate, frames=int(var.shape[0]/plot_every))
# anim.save("churn_and_restore.gif")


In [ ]:
##### PLOT CONTOUR OF PSI AT FIRST AND LAST TIMESTAMP #####
fig,ax = plt.subplots(1)
levels = 20
v = "psi"
t1 = 0
t2 = -1
ax.contour(ds[x],ds[y],ds[v][t1].values.T, levels=levels)
ax.contour(ds[x],ds[y],ds[v][t2].values.T, linestyles="--", levels=levels)

In [ ]:
##### PLOT CONSERVED QUANTITIES ######
def integrate_dxdy(ds, variable):
    I = np.zeros(len(ds.t))
    for t in range(len(ds.t)):
        I[t] = np.sum(ds.dx * ds.dy * variable[t])
    return I

def get_tot_pol_flux(ds):
    # [psi_0 a_mid^2]
    tot_pol_flux = integrate_dxdy(ds, ds["psi"])
    return tot_pol_flux

def get_tot_pressure(ds):
    # [P_0 a_mid^2]
    a_mid = ds.metadata["a_mid"]
    P_0 = ds.metadata["P_0"]
    tot_pressure = P_0 * a_mid**2 * integrate_dxdy(ds, ds["P"]) 
    return tot_pressure

def get_tot_energy(ds):
    # [P_0 a_mid^2]

    ddx_psi = ds["psi"].differentiate("x")
    ddy_psi = ds["psi"].differentiate("y")

    beta_p = ds.metadata["beta_p"]
    epsilon = ds.metadata["epsilon"]
    R_0 = ds.metadata["R_0"]
    a_mid = ds.metadata["a_mid"]
    psi_0 = ds.metadata["psi_0"]
    P_0 = ds.metadata["P_0"]
    B_pmid = ds.metadata["B_pmid"]

    x_c = (ds["x"] - 0.5*np.sum(ds["dx"].isel(x=0)))
    B_px = ddx_psi / (1 + epsilon * x_c)
    B_py = ddy_psi / (1 + epsilon * x_c)
    P = ds["P"]
    u_x = ds["u_x"]
    u_y = ds["u_y"]
    mu_0 = 1.256637e-6
    
    mag_energy = 0.2*(P_0/beta_p) * a_mid**2 * integrate_dxdy(ds, B_px**2 + B_py**2) 
    kin_energy = 0.5 * a_mid ** 2 * P_0 * integrate_dxdy(ds, u_x**2 + u_y**2) 
    pot_energy = -P_0 * epsilon * a_mid **2 * integrate_dxdy(ds, P * x_c)
    
    return mag_energy/(P_0 * a_mid**2), kin_energy/(P_0 * a_mid**2), pot_energy/(P_0 * a_mid**2)

nxg = 3
nx = len(ds.x)
ds_trimmed = ds.isel(x=range(nxg,nx-nxg))
tot_pol_flux = get_tot_pol_flux(ds_trimmed)
tot_pressure = get_tot_pressure(ds_trimmed)
M, K, Pi = get_tot_energy(ds_trimmed)

fig,ax = plt.subplots(4, sharex=True)
ax[0].plot(ds_trimmed[r"t"], tot_pol_flux)
ax[0].set_title(r"$\psi$")
ax[0].set_ylabel(r"[$\psi_0 a_{mid}^2$]")
ax[1].plot(ds_trimmed[r"t"], tot_pressure)
ax[1].set_title(r"$P$")
ax[1].set_ylabel(r"[$P_0 a_{mid}^2$]")
ax[3].plot(ds_trimmed[r"t"], M - M[0], "-", label=r"$M(t)-M(t=0)$")
ax[3].plot(ds_trimmed[r"t"], K - K[0], "-", label=r"$K(t)-K(t=0)$")
ax[3].plot(ds_trimmed[r"t"], Pi - Pi[0], "-", label=r"$\Pi(t)-\Pi(t=0)$")
ax[3].plot(ds_trimmed[r"t"], (M-M[0]) + (K-K[0]) + (Pi-Pi[0]), "--", color=r"black", label=r"$E(t)-E(t=0)$")
ax[3].set_ylabel(r"[$P_0 a_{mid}^2$]")
ax[2].plot(ds_trimmed[r"t"], (M) + (K) + (Pi), "-", color=r"black")
ax[2].set_title(r"$E=M+K+\Pi$")
ax[2].set_ylabel(r"[$P_0 a_{mid}^2$]")
# ax[3].set_title("$E=M+K+\Pi$")
[ax[i].legend(loc="lower left") for i in range(len(ax))]
ax[-1].set_xlabel("$t\ [t_0]$")
[a.grid() for a in ax]
fig.tight_layout()

# Staggered

In [ ]:
ds.bout.animate_list(["omega", "P", "psi", "phi", "u_x", "u_y"])

In [ ]:
##### ANIMATE MIXED COLOURMAPS AND LINE PLOTS #####
offset = -28
ds.bout.animate_list(["phi",
                      "omega",
                      ds["u_x"].isel(y=32+offset), 
                      ds["u_x"].isel(x=35+offset),
                      ds["u_ys"].isel(y=32+offset), 
                      ds["u_ys"].isel(x=35+offset),
                      ],
                      ncols=2,
                    )
